1. Duplicates analysis
2. Nulls
3. Dates analysis: (outliers,etc)
4. Orders analysis (outliers,etc)

In [63]:
# Importing packages:

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
from collections import Counter
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import math
import seaborn as sns


%matplotlib inline
pd.options.display.max_columns = None

In [64]:
# Defining the search path of the file, the name and the separator:

file_path = "../../data/01_raw/"
file_name = "b2-transactions.csv" #'b2-transactions_sample.csv' 
exit_path = "../../data/02_intermediate/"

filtered_file_name="c1-filtered_transactions.csv"

sep=";"

In [65]:
target_products_file_name = exit_path + 'filtered_transactions_not_clean.csv' 


In [66]:
df_transactions_filtered = pd.read_csv(df_target_products_file_name, sep = sep)

In [67]:
df_transactions_filtered.head()

,product_id,description,order_date,section,store,units_ordered,desc_normalized,target_names_prod_by_prod
0,100.0,CROISANTS,1/1/2008 0:00:00,0,VeUp,"20,00",croissant,croissant simple
1,182.0,PALMERAS DE TRUFA,1/1/2008 0:00:00,0,VeUp,"4,00",palmera de trufa,palmera chocolate
2,101.0,CROISSANT,1/1/2008 0:00:00,0,VeUp,"0,00",croissant,croissant simple
3,103.0,CROISSANT PETIT,1/1/2008 0:00:00,0,VeUp,"18,00",croissant petit,croissant petit
4,112.0,TORTELES,1/1/2008 0:00:00,0,VeUp,"60,00",torteles,tortel


In [68]:
df_transactions_filtered.shape

(967993, 8)

In [150]:
df_transactions_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 967993 entries, 0 to 967992
Data columns (total 8 columns):
product_id                   967993 non-null float64
description                  967993 non-null object
order_date                   967993 non-null object
section                      967993 non-null int64
store                        967993 non-null object
units_ordered                967679 non-null object
desc_normalized              967993 non-null object
target_names_prod_by_prod    967993 non-null object
dtypes: float64(1), int64(1), object(6)
memory usage: 59.1+ MB


"order_date" should be date-type but is not, and "units_ordered" should be numeric, but they are not, and there are some nulls... Lets treat the nulls before parsing to the right types. 

# NULLs Analysis


In [153]:
df_transactions_filtered.isnull().sum()

product_id                     0
description                    0
order_date                     0
section                        0
store                          0
units_ordered                314
desc_normalized                0
target_names_prod_by_prod      0
dtype: int64

As we saw before, there are only nulls in "units_ordered". The good news is that it does seem to be many of them, lets have a look at it before we decide what to do to them:

In [156]:
df_transactions_filtered[df_transactions_filtered.isnull()]

OptionError: "No such keys(s): 'compute.use_numexpr'"

# Duplicates Analysis

Lets see if we have any duplicates:

In [69]:
df_transactions_filtered.duplicated().sum()

120336

Wow! this was unexpected 120.336 duplicates... We need to have a deeper look:

In [147]:
df_transactions_filtered[df_transactions_filtered.duplicated()]

,product_id,description,order_date,section,store,units_ordered,desc_normalized,target_names_prod_by_prod
261,182.0,PALMERAS DE TRUFA,1/1/2009 0:00:00,0,ZiUO,"0,00",palmera de trufa,palmera chocolate
292,182.0,PALMERAS DE TRUFA,1/1/2009 0:00:00,0,AeUP,"0,00",palmera de trufa,palmera chocolate
325,182.0,PALMERAS DE TRUFA,1/1/2009 0:00:00,0,ViUP,"0,00",palmera de trufa,palmera chocolate
347,182.0,PALMERAS DE TRUFA,1/1/2009 0:00:00,0,LiUP,"0,00",palmera de trufa,palmera chocolate
358,182.0,PALMERAS DE TRUFA,1/1/2009 0:00:00,0,GeUP,"0,00",palmera de trufa,palmera chocolate
387,182.0,PALMERAS DE TRUFA,1/1/2009 0:00:00,0,PoUP,"0,00",palmera de trufa,palmera chocolate
432,182.0,PALMERAS DE TRUFA,1/1/2010 0:00:00,0,GrUP,"0,00",palmera de trufa,palmera chocolate
434,462.0,MANZANA 2º,1/1/2010 0:00:00,0,LiUP,"0,00",manzana 2,tarta de manzana
437,182.0,PALMERAS DE TRUFA,1/1/2010 0:00:00,0,LiUP,"0,00",palmera de trufa,palmera chocolate
438,182.0,PALMERAS DE TRUFA,1/1/2010 0:00:00,0,LiUP,"0,00",palmera de trufa,palmera chocolate


Interesting, it seems that all have 0 units ordered, but lets dig more into this:

In [148]:
df_transactions_filtered[df_transactions_filtered.duplicated()]["units_ordered"].value_counts()

0,00     118007
1,00       1583
2,00        255
6,00         71
3,00         71
4,00         65
8,00         50
10,00        38
5,00         32
12,00        21
15,00        18
20,00        12
25,00        12
7,00         12
18,00        12
50,00         9
30,00         8
14,00         7
16,00         7
40,00         5
9,00          4
11,00         4
24,00         3
13,00         3
26,00         2
55,00         2
35,00         2
22,00         2
45,00         1
0,50          1
64,00         1
60,00         1
90,00         1
70,00         1
Name: units_ordered, dtype: int64

Unfortunately, it seems that not all the duplicates are 0, otherwise we could have deleted the duplicates as they don't add any value. However we now need to find out a bit more the ones greater than 0.

In [149]:
df_transactions_filtered["units_ordered"].astype(float)

ValueError: could not convert string to float: '1,00'

In [125]:
aux2 = aux[aux["units_ordered"] == "70,00"]

In [131]:
aux2

,product_id,description,order_date,section,store,units_ordered,desc_normalized,target_names_prod_by_prod
396521,112.0,TORTELES,20/2/2016 0:00:00,0,VeUp,"70,00",torteles,tortel


In [133]:
df_transactions_filtered == aux2

ValueError: Can only compare identically-labeled DataFrame objects

Lets check the original file

In [71]:
original_file = file_path + "b2.txt"

# Column names
col_names = ["product_id",
          "descrption",
          "order_date",
          "section",
          "BmUP",
          "VeUp",
          "CzUP",
          "AaUP",
          "SeUP",
          "JPUP",
          "RzUP",
          "AeUP",
          "LiUP",
          "EnUP",
          "GoUP",
          "GrUP",
          "AnUP",
          "MoUP",
          "PoUP",
          "MsUP",
          "ViUP",
          "ZiUO",
          "GeUP"]

In [77]:
order_datedf_original = pd.read_csv(original_file, sep = sep, encoding = "ISO-8859-1",parse_dates = ["order_date"], names = col_names,infer_datetime_format=True )

In [78]:
df_original.head()

,product_id,description,order_date,section,BmUP,VeUp,CzUP,AaUP,SeUP,JPUP,RzUP,AeUP,LiUP,EnUP,GoUP,GrUP,AnUP,MoUP,PoUP,MsUP,ViUP,ZiUO,GeUP
0,211.0,EMPANADILLAS SALMON,12/6/2009 0:00:00,0,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
1,1231.0,MALLORCA CELOFAN VAINILLA,13/6/2009 0:00:00,0,"0,00","0,00","1,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
2,1233.0,ROCAS FILET VAINILLA,13/6/2009 0:00:00,0,"1,00","0,00","0,00","0,00","1,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
3,1234.0,LICOR GUINDAS,13/6/2009 0:00:00,0,"1,00","1,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
4,1235.0,LICOR UVAS,13/6/2009 0:00:00,0,"1,00","0,00","1,00","0,00","1,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"


In [138]:
df_original[(df_original["order_date"]=="1/1/2009 0:00:00")&(df_original["description"].str.contains("PALMERAS DE TRUFA"))]

,product_id,description,order_date,section,BmUP,VeUp,CzUP,AaUP,SeUP,JPUP,RzUP,AeUP,LiUP,EnUP,GoUP,GrUP,AnUP,MoUP,PoUP,MsUP,ViUP,ZiUO,GeUP
774282,182.0,PALMERAS DE TRUFA,1/1/2009 0:00:00,0,"2,00","5,00","2,00","2,00","8,00","3,00","2,00","0,00","0,00","3,00","2,00","4,00","4,00","3,00","0,00","0,00","0,00","0,00","0,00"
781175,182.0,PALMERAS DE TRUFA,1/1/2009 0:00:00,0,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","5,00","0,00","0,00","0,00"


In [139]:
df_original.duplicated().sum()

5282

In [140]:
df_original[(df_original["order_date"]=="20/2/2016 0:00:00")&(df_original["description"].str.contains("TORTELES"))]

,product_id,description,order_date,section,BmUP,VeUp,CzUP,AaUP,SeUP,JPUP,RzUP,AeUP,LiUP,EnUP,GoUP,GrUP,AnUP,MoUP,PoUP,MsUP,ViUP,ZiUO,GeUP
1462852,112.0,TORTELES,20/2/2016 0:00:00,0,"40,00","70,00","35,00","60,00","36,00","30,00","16,00","0,00","0,00","30,00","0,00","0,00","20,00","20,00","30,00","0,00","0,00","0,00","17,00"
1462883,112.0,TORTELES,20/2/2016 0:00:00,0,"0,00","70,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
